# Notebook 12: Plotting small circles on the stereonet

A plane that does not pass through the center of the sphere, intersects the sphere along a small circle. Small circles can also be plotted on the stereonet, by rotating incrementally a line. To draw a small circle, we just need to rotate a line about the axis of the small circle in fixed increments (e.g., 1°) until completing 360°. Any line making an angle less than 90° with the axis of rotation will trace a cone, which will plot on the stereonet as a small circle.

## Python function

For a small circle with an axis of a given orientation (trend and plunge) and apical angle, the following function returns the coordinates of the small circle on an equal angle or equal area stereonet.

In [1]:
# this makes visible our functions folder
import sys, os
sys.path.append(os.path.abspath("../functions"))

In [2]:
# small_circle function: this function is also in functions/small_circle.py

import numpy as np
from rotate import rotate
from st_coord_line import st_coord_line
from zero_twopi import zero_twopi

def small_circle(trda,plga,cangle,stype):
	"""
	small_circle computes the paths of a small circle defined
	by its axis and cone angle, on an equal angle or equal
	area stereonet of unit radius
	
	trda = trend of axis
	plga = plunge of axis
	cangle = cone angle
	stype = Stereonet type: 0 = equal angle, 1 = equal area
	path1 and path2 = vectors with the x and y coordinates
		of the points in the small circle paths
	np1 and np2 = Number of points in path1 and path2
	
	NOTE: All angles should be in radians
	
	Python function translated from the Matlab function
	SmallCircle in Allmendinger et al. (2012)
	"""
	pi = np.pi
	# Find where to start the small circle
	if (plga - cangle) >= 0.0:
		trd = trda
		plg = plga - cangle
	else:
		if plga == pi/2.0:
			plga = plga * 0.9999
		angle = np.arccos(np.cos(cangle)/np.cos(plga))
		trd = zero_twopi(trda+angle)
		plg = 0.0
	
	# To make the small circle, rotate the starting line
	# 360 degrees in increments of 1 degree
	rot = np.radians(np.arange(0,361,1))
	path1 = np.zeros((rot.shape[0],2))
	path2 = np.zeros((rot.shape[0],2))
	np1 = np2 = 0
	for i in range(rot.size):
		# Rotate line: The line is considered as a vector
		rtrd , rplg = rotate(trda,plga,rot[i],trd,plg,"v")
		# Calculate stereonet coordinates and add to path
		# If rotated plunge is positive add to 1st path
		if rplg >= 0.0:
			path1[np1,0] , path1[np1,1] = st_coord_line(rtrd,rplg,stype)
			np1 += 1
		# Else add to 2nd path
		else:
			path2[np2,0] , path2[np2,1] = st_coord_line(rtrd,rplg,stype)
			np2 += 1
	
	return path1, path2, np1, np2

## Application

Now let's plot a small circle on an equal angle and an equal area stereonet. The code below plots the stereonets side by side and allows setting the trend and plunge of the axis, and the apical angle interactively using sliders.

In [3]:
def plot_small_circle(trend, plunge, apical_angle):
    """
    Plots a small circle on an equal angle (left) and equal area (right) stereonet

    Input:
    trend = trend of axis
    plunge = plunge of axis
    apical_angle = cone angle

    Output:
    None
    """
    # import necessary libraries
    import matplotlib.pyplot as plt

    # create a figure with two subplots side by side
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # coordinates of primitive circle, radius = 1
    theta = np.radians(np.arange(0, 361, 1))
    x_c = np.cos(theta)
    y_c = np.sin(theta)

    # plot the primitive circle and the line
    for i in range(2):
        # primitive and center
        ax[i].plot(x_c, y_c, 'k')
        ax[i].plot(0, 0, 'k+')
        
        # small circle
        path1, path2, np1, np2 = small_circle(np.radians(trend), 
                                              np.radians(plunge), np.radians(apical_angle), i)
        ax[i].plot(path1[:np1,0], path1[:np1,1], 'r-')
        if np2 > 0:
            ax[i].plot(path2[:np2,0], path2[:np2,1], 'r-')
        
        # axes settings
        ax[i].set_aspect('equal')
        ax[i].set_xlim(-1.1, 1.1)
        ax[i].set_ylim(-1.1, 1.1)
        ax[i].axis('off')
        ax[i].set_title(['Equal angle', 'Equal area'][i] + ' stereonet')
    
    plt.show()

# use ipywidgets to interact with the function
from ipywidgets import interact
interact(plot_small_circle, trend=(0, 360, 1), plunge=(0, 90, 1), apical_angle=(0, 90, 1));

interactive(children=(IntSlider(value=180, description='trend', max=360), IntSlider(value=45, description='plu…

As you can see, on the equal angle stereonet a small circle plots as a circle everywhere, but its size increases with decreasing axis plunge. On the equal area stereonet, a small circle does not plot as a circle everywhere but just when the axis is vertical; the size of the small circle howewer is everywhere the same.